In [1]:
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multivariate_normal
import pandas as pd
from scipy.stats import truncnorm

In [2]:
df = pd.read_csv("data/data/bank-note/train.csv", header=None)
d = df.to_numpy()
X = d[:,:-1]
x = np.hstack((X,np.ones((X.shape[0],1))))
y = d[:,-1] 
# y = np.expand_dims(d[:,-1], axis=-1) 

x.shape, y.shape

((872, 5), (872,))

In [3]:
df = pd.read_csv("data/data/bank-note/test.csv", header=None)
d = df.to_numpy()
x_test = d[:,:-1]
x_test = np.hstack((x_test,np.ones((x_test.shape[0],1))))

y_test = d[:,-1]
# y_test = np.expand_dims(d[:,-1], axis=-1) 

x_test.shape, y_test.shape

((500, 5), (500,))

# Part a

In [18]:
def get_output(weight, data, regression= "logistic"):
    """
        Output of different regression. Taken from Assignment 2.
        returns #examples x 1 arrays
    """
    dot_product = np.matmul(data,weight)
    if regression == "logistic":
        output = get_sigmoid(dot_product)
    elif regression == "probit":
        output = norm.cdf(dot_product)
    elif regression == "multiclass":
        output = softmax(dot_product, axis=1)

    return output, dot_product

def get_log_likelihood(phi, pred, t, dot_product, weight, reg= 1):
    """
        Returns log likelihood of the logistic regression
        t = N x 1
    """
    prior = -0.5* np.sum(np.multiply(weight, weight))
    likelihood = np.multiply(t, np.log(pred+TOLERANCE)) + np.multiply(1.0- t, np.log(1.0-pred+TOLERANCE))
    likelihood = np.sum(likelihood)

    return prior + likelihood

def get_sigmoid(x):
    """
        Numerically stable version of sigmoid function. Taken from Assignment 2.
    """  
    output = np.zeros(x.shape)
    ind1 = (x >= 0)
    ind2 = (x  < 0)
    output[ind1] = 1 / (1 + np.exp(-x[ind1]))
    output[ind2] = np.divide(np.exp(x[ind2]), (1 + np.exp(x[ind2])))

    return output

def get_gradient(phi, pred, t, dot_product, weight, reg= 1, regression= "logistic"):
    """
        Returns log likelihood of the logistic regression. Taken from Assignment 2
        t = (N, 1)
        weight = (D, 1)
    """
    if regression == "logistic":
        gradient = np.matmul(phi.T, pred - t)
    elif regression == "probit":
        R = np.eye(pred.shape[0])
        for i in range(pred.shape[0]):
            y_n  = pred[i,0]
            dotp = dot_product[i, 0]
            pdf  = norm.pdf(dotp)
            R[i,i] = pdf/(y_n*(1-y_n) + TOLERANCE)
        gradient = np.matmul(np.matmul(phi.T, R), pred-t)
    elif regression == "multiclass":
        gradient = np.matmul(phi.T, pred - t)

    # Add regularization
    gradient += weight/ reg
    return gradient

def get_KE(p, scale= 1):
    """ 
        Returns KE from the momentum vector
    """
    p = p.flatten()
    return scale * 0.5*np.sum(np.multiply(p, p))

def to_accept_without_log(x, x_new):
    """
        Acceptance rule without any log. 
    """
    if x_new>x:
        return True
    else:
        accept=np.random.uniform(0,1)
        return (accept < x_new/(x+TOLERANCE))

def hybrid_monte_carlo(train_data, train_label, z_init, num_iterations, epsilon, num_leapfrog_steps, collect_final_sample_frequency= 10, display_frequency= 5000, scale_KE= 1):
    """
        Gets posterior samples for Bayes Logistic Regression using HMC algorithm
        z_int= (dim, 1)
    """
    dim = train_data.shape[1]
    z = z_init

    accepted = [] # Keeps track of accepted samples
    sampled  = [] # Keeps track of all samples
    final    = [] # Keeps track of final samples which are sampled in a cyclic manner
    
    for i in range(num_iterations):
        # Old energy = -loglik and Old gradient
        pred, dot_product = get_output(z, train_data)
        old_PE   =  -get_log_likelihood(phi= train_data, pred= pred, t= train_label[:, np.newaxis], dot_product= dot_product, weight= z)
        
        # There is no minus since gradient function returns gradient of negative log likelihood
        old_grad =  get_gradient(phi= train_data, pred= pred, t= train_label[:, np.newaxis], dot_product= dot_product, weight= z)

        new_z = np.copy(z)              # deep copy of array
        new_grad  = np.copy(old_grad)   # deep copy of array

        # draw random momentum vector from unit Gaussian which decides the energy
        # given out for exploration
        p = np.random.normal(0.0, 1.0, (dim, 1))

        # Compute Hamiltonian
        H = get_KE(p, scale= scale_KE) + old_PE

        # Suggest new candidate using gradient + Hamiltonian dynamics.
        # Leapfrog
        for j in range(num_leapfrog_steps):  
            # Make first half step in p, full step in z and then again half step in p
            p        -= (epsilon/2.0)*new_grad
            new_z    += epsilon*p
            pred, dot_product = get_output(new_z, train_data)
            new_grad  = get_gradient(phi= train_data, pred= pred, t= train_label[:, np.newaxis], dot_product= dot_product, weight= new_z)
            p        -= (epsilon/2.0)*new_grad

        # Compute new Hamiltonian
        pred, dot_product = get_output(new_z, train_data)
        new_PE = -get_log_likelihood(phi= train_data, pred= pred, t= train_label[:, np.newaxis], dot_product= dot_product, weight= new_z)
        new_H  = get_KE(p, scale= scale_KE) + new_PE
        
        sampled.append(new_z)
        
        # Accept new candidate in Monte-Carlo fashion.
        if to_accept_without_log(get_prob_from_energy(H), get_prob_from_energy(new_H)):            
            z = new_z
            accepted.append(new_z)

        if i % collect_final_sample_frequency == 0:
            # Sample from the current parameters
            final.append(z)

        if (i+1) % display_frequency == 0 or i == num_iterations-1:
            print("Iter {:6d} done".format(i+1))
    
    return np.array(accepted), np.array(sampled), np.array(final), z

def get_prob_from_energy(energy):
    return np.exp(-energy)

def get_accuracy(pred, test_label, regression= "logistic"):
    """
        Gets accuracy in % for predictions. Taken from Assignment 2.
    """
    if regression == "multiclass":
        pred_max = np.argmax(pred, axis=1)
        gt_max   = np.argmax(test_label, axis=1)
        acc = np.sum(pred_max == gt_max)*100.0/pred.shape[0]
    elif regression == "logistic" or regression == "probit":
        if pred.ndim == 2:
            pred = pred[:,0]
        pred[pred >= 0.5] = 1.0
        pred[pred <  0.5] = 0.0
    acc = np.sum(pred == test_label)*100.0/pred.shape[0]

    return acc

def get_prediction_likelihood_without_complications(test_data, test_label, weight):
    """
        Returns prediction likelihood on a sample weight without using any hessian
        test_data  = N x D
        test_label = N 
        weight     = D x 1
    """
    pred, _ = get_output(weight, test_data)
    pred = pred[:,0]
    pred_like = np.multiply(test_label, np.log(pred + TOLERANCE)) + np.multiply(1.0-test_label, np.log(1.0-pred+ TOLERANCE))
    return np.exp(np.mean(pred_like))

def test_on_posterior(test_data, test_label, posterior_samples):
    """
        Returns stats on posterior samples
    """
    print("Testing on posterior samples...")
    num_posterior_samples = posterior_samples.shape[0]
    avg_pred_test    = np.zeros((num_posterior_samples, ))
    avg_pred_log_lld = np.zeros((num_posterior_samples, ))
                    
    for k in range(num_posterior_samples):
        # Use the posterior samples
        w_sampled = posterior_samples[k]
        
        # Get the hessian
        #pred, dot_product = get_output(w_sampled, train_data)
        #hessian  = get_hessian (phi= train_data, pred= pred[:, np.newaxis], t= train_label[:, np.newaxis], dot_product= dot_product)
        
        pred_test, _         = get_output  (w_sampled, test_data)
        acc                  = get_accuracy(pred_test, test_label) 
        pred_likelihood      = get_prediction_likelihood_without_complications(test_data, test_label, w_sampled) #get_prediction_likelihood(test_data, test_label, w_sampled, hessian)
        avg_pred_test[k]     = acc
        avg_pred_log_lld [k] = np.log(pred_likelihood)
        
        if (k+1)%100 == 0 or k== num_posterior_samples-1:
            print("{:5d} Posterior Weight samples Test_data Pred_acc= {:.2f}, Pred_log_likelihood= {:.2f}".format(k+1, np.mean(avg_pred_test[:k]), np.mean(avg_pred_log_lld[:k])))    

In [19]:
dim = x.shape[1]

num_iterations       = 100000#//10
num_iterations_final = 10000 #//10
collect_final_sample_frequency = 10
display_frequency    = 5000
TOLERANCE = 1e-5

print("\n=======================================================================")
print("\tHamiltonian Monte Carlo Sampling with Leapfrog")
print("=======================================================================")
epsilon_array = np.array([0.005, 0.01, 0.02, 0.05])
num_leapfrog_steps_array = np.array([10, 20, 50])

for i in range(epsilon_array.shape[0]):
    for j in range(num_leapfrog_steps_array.shape[0]):
        epsilon            = epsilon_array[i]
        num_leapfrog_steps = num_leapfrog_steps_array[j]
        print("\nBurnin stage, epsilon = {:.3f}, L= {}".format(epsilon, num_leapfrog_steps))
        w_init = np.zeros((dim, 1))
        _, _, _, w_new = hybrid_monte_carlo(x, y, z_init= w_init, num_iterations= num_iterations, 
                                            epsilon= epsilon, num_leapfrog_steps= num_leapfrog_steps, 
                                            collect_final_sample_frequency= collect_final_sample_frequency, 
                                            scale_KE= 1)

        # Remember to initialize from new values
        print("Generating samples after burnin stage...")
        accepted, sampled, posterior_samples, _ = hybrid_monte_carlo(x, y, z_init= w_new , 
                                                                     num_iterations= num_iterations_final, 
                                                                     epsilon= epsilon, 
                                                                     num_leapfrog_steps= num_leapfrog_steps, 
                                                                     collect_final_sample_frequency= collect_final_sample_frequency,
                                                                     scale_KE= 1)
        acceptance_rate = accepted.shape[0]/sampled.shape[0]
        test_on_posterior(x_test, y_test, posterior_samples)
        print("Acceptance rate= {:2f}".format(acceptance_rate))


	Hamiltonian Monte Carlo Sampling with Leapfrog

Burnin stage, epsilon = 0.005, L= 10
Iter   5000 done
Iter  10000 done
Iter  15000 done
Iter  20000 done
Iter  25000 done
Iter  30000 done
Iter  35000 done
Iter  40000 done
Iter  45000 done
Iter  50000 done
Iter  55000 done
Iter  60000 done
Iter  65000 done
Iter  70000 done
Iter  75000 done
Iter  80000 done
Iter  85000 done
Iter  90000 done
Iter  95000 done
Iter 100000 done
Generating samples after burnin stage...
Iter   5000 done
Iter  10000 done
Testing on posterior samples...
  100 Posterior Weight samples Test_data Pred_acc= 99.00, Pred_log_likelihood= -0.04
  200 Posterior Weight samples Test_data Pred_acc= 99.00, Pred_log_likelihood= -0.04
  300 Posterior Weight samples Test_data Pred_acc= 99.00, Pred_log_likelihood= -0.04
  400 Posterior Weight samples Test_data Pred_acc= 99.00, Pred_log_likelihood= -0.04
  500 Posterior Weight samples Test_data Pred_acc= 99.00, Pred_log_likelihood= -0.04
  600 Posterior Weight samples Test_data 

Iter   5000 done
Iter  10000 done
Iter  15000 done
Iter  20000 done
Iter  25000 done
Iter  30000 done
Iter  35000 done
Iter  40000 done
Iter  45000 done
Iter  50000 done
Iter  55000 done
Iter  60000 done
Iter  65000 done
Iter  70000 done
Iter  75000 done
Iter  80000 done
Iter  85000 done
Iter  90000 done
Iter  95000 done
Iter 100000 done
Generating samples after burnin stage...
Iter   5000 done
Iter  10000 done
Testing on posterior samples...
  100 Posterior Weight samples Test_data Pred_acc= 98.95, Pred_log_likelihood= -0.03
  200 Posterior Weight samples Test_data Pred_acc= 98.95, Pred_log_likelihood= -0.03
  300 Posterior Weight samples Test_data Pred_acc= 98.95, Pred_log_likelihood= -0.03
  400 Posterior Weight samples Test_data Pred_acc= 98.96, Pred_log_likelihood= -0.03
  500 Posterior Weight samples Test_data Pred_acc= 98.96, Pred_log_likelihood= -0.03
  600 Posterior Weight samples Test_data Pred_acc= 98.96, Pred_log_likelihood= -0.03
  700 Posterior Weight samples Test_data Pr